In [ ]:
# Allow src folder to be imported from this notebook
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

# ILP Formulation to cover an MSA with Blocks
___

- [X] Select minimum set of blocks covering the MSA
- [X] Create a graph from the selected blocks
- [X] Visualize the graph
- [X] Output the graph in gfa format
- [ ] join not covered positions by maximal blocks (no trivial blocks unless is needed)

In [ ]:
import json
import gurobipy as gp
from gurobipy import GRB
from src.blocks import Block
from src.msa import AnalyzerMSA

import numpy as np
from collections import defaultdict

# pangenome graph
from src.graph import (
    nodes_edges_from_blocks, 
    PlotGraph
)
from src.graph.bandage_labels_from_gfa import bandage_labels

In [ ]:
# MSA
NAME_MSA = "toyexample"
amsa = AnalyzerMSA()
path_msa = f"../msas/{NAME_MSA}.fa"
align, n_seqs, n_cols = amsa.load_msa(path_msa)
n_seqs, n_cols

In [ ]:
str(align[0,1:2].seq)

**Blocks for each position (r,c) in the MSA** 

$({r},c,c,MSA[r,c])$

In [ ]:
blocks_one_char = []
for col in range(n_cols):
    for row in range(n_seqs):
        blocks_one_char.append(
            Block(K=(row,), i=col, j=col, label=align[row,col])
        )

blocks_one_char[:3]

**Set of blocks from the  decomposition**

In [ ]:
# Load set of decomposed blocks
path_blocks = f"../experiment/block_decomposition/{NAME_MSA}.json"

with open(path_blocks) as fp:
    decomposed_blocks = [Block(*block) for block in json.load(fp)] 

decomposed_blocks[:3]

In [ ]:

def check_coverage_panel(n_seqs, n_cols, blocks):
    """returns a matrix of size equal to msa (n_seq x n_cols) with 
    the number of blocks in the list_blocks that covers each position"""

    # coverage_by_pos = defaultdict(int)
    coverage_by_pos = np.zeros((n_seqs, n_cols))
    for block in blocks:
        for r in block.K:
            for c in range(block.i,block.j+1):
                coverage_by_pos[r,c] += 1
    return coverage_by_pos
    
import numpy as np 
coverage_by_pos = check_coverage_panel(n_seqs, n_cols, decomposed_blocks)
coverage_by_pos

def get_missing_blocks(coverage_by_pos): 
    """return the missing blocks to cover the MSA
    all consecutives one character not covered positions are
    clustered in one block
    """
    rows, cols=np.where(coverage_by_pos == 0)
    missing_blocks = [(r,c) for r,c in zip(rows,cols)]
    # missing_blocks = get_missing_blocks(coverage_by_pos)
    missing_blocks = sorted(missing_blocks, key= lambda d: (d[0],d[1]))
    idx_missing_blocks_by_seq = defaultdict(list)
    for pos in missing_blocks: 
        idx_missing_blocks_by_seq[pos[0]].append(pos[1])
    # return get_missing_blocks
    return idx_missing_blocks_by_seq


def get_consecutive_pos(positions: list[int]):
    "given a list with positions, split it in sublists of consecutive numbers"
    rv = []

    # Set up current list with first element of input
    curr = [positions[0]]

    # For each remaining element:
    for x in positions[1:]:
        # If the next element is not 1 greater than the last seen element
        if x - 1 != curr[-1]:
            # Append the list to the return variable and start a new list
            rv.append(curr)
            curr = [x]
        # Otherwise, append the element to the current list.
        else:
            curr.append(x)
    rv.append(curr)
    return rv

# now split each row into separate sublist of (row,col)
missing_blocks=[]
idx_missing_blocks_by_seq = get_missing_blocks(coverage_by_pos)
for seq, cols in idx_missing_blocks_by_seq.items():
    consecutive_pos = get_consecutive_pos(cols)
    for pos in consecutive_pos: 
        label = str(align[int(seq)].seq[pos[0]:pos[-1]+1]) #if pos[0]!=pos[-1] else align[seq,pos[0]]
        missing_blocks.append(
        Block(K=(seq,), i=pos[0],j=pos[-1], label=label)
        )

missing_blocks

In [ ]:
# set B: input blocks (maximal blocks, the decompositions under intersection by pairs and blocks of one position in the MSA)
set_B = decomposed_blocks + blocks_one_char + [block for block in missing_blocks if block.j-block.i+1 > 1]

# set_B = blocks_one_char
# set_B.extend([Block((0,),0,0,"A"), Block((2,),4,4,"A")])

# write idx for blocks 
blocks = [(",".join([str(r) for r in b.K]), b.i, b.j) for b in set_B] # (K,i,j)

# write idx for MSA positions (row, col)
msa_positions = [(r,c) for r in range(n_seqs) for c in range(n_cols)] 

# dictionary to store the string of each block indexed as Gurobipy uses it
strings_ = { (",".join([str(_) for _ in b.K]), b.i, b.j): b.label for b in set_B}

In [ ]:
def check_intersection(block1,block2):
    "intersection of blocks "
    block1_K = block1[0].split(",")
    block2_K = block2[0].split(",")

    # check for not empty intersection, otherwise skip to the next block1 in the list
    common_rows = list(set(block1_K).intersection(set(block2_K))) # intersection set K
    common_cols = list(set(range(block1[1],block1[2]+1)).intersection(set(range(block2[1],block2[2]+1)))) # intersection columns [i,j]

    return True if common_rows and common_cols else False

# example
block1 = ("1,2",1,1)
block2 = ("3",1,3)
check_intersection(block1, block2)

In [ ]:
# Create the model
model = gp.Model("pangeblocks")

# define variables
C = model.addVars(blocks, vtype=GRB.BINARY, name="C")
U = model.addVars(msa_positions, vtype=GRB.BINARY, name="U")

# Constraints: 
for r,c in msa_positions:

    # subset of blocks that covers the position [r,c]
    subset_C = [ C[K,i,j] for K,i,j in blocks if str(r) in K.split(",") and i<=c<=j ]
    if (r,c) in [(0,0),(2,4)]:
        print(f"{(r,c)}:len {len(subset_C)}")
    if len(subset_C)>0:
        # print(f"{len(subset_C)} blocks cover the position {(r,c)}")
        
        ## 1. each position in the MSA is covered ONLY ONCE
        model.addConstr( U[r,c] <= sum(subset_C), name=f"constraint1({r},{c})")
        
        ## 2. each position of the MSA is covered AT LEAST by one block
        model.addConstr( U[r,c] >= 1, name=f"constraint2({r},{c})")


## 3. overlapping blocks cannot be chosen
# sort all blocks, 
blocks = sorted(blocks, key=lambda b: b[1]) # sort blocks by the starting position (K,start,end)

# and analyze the intersections while update the constraints
names_constraint3=[]
for pos1,block1 in enumerate(blocks[:-1]):
    # compare against the next blocks in the sorted list
    for rel_pos, block2 in enumerate(blocks[pos1+1:]):
        pos2 = rel_pos + pos1 + 1
        block2 = blocks[pos2]
        
        # check for not empty intersection, otherwise, skip to the next block  
        # note: set K is a string with the rows concatenated by a "," (due to Gurobi requirements to index the variables)
        block1_K = block1[0].split(",")
        block2_K = block2[0].split(",")

        # check for not empty intersection, otherwise skip to the next block1 in the list
        common_rows = list(set(block1_K).intersection(set(block2_K))) # intersection set K
        common_cols = list(set(range(block1[1],block1[2]+1)).intersection(set(range(block2[1],block2[2]+1)))) # intersection columns [i,j]

        if (common_rows and common_cols):
            
            # if the blocks intersect, then create the restriction 
            K1,i1,j1=block1
            K2,i2,j2=block2
            name_constraint=f"constraint3({K1},{i1},{j1})-({K2},{i2},{j2})"
            model.addConstr(C[block1] + C[block2] <= 1 , name=name_constraint)
            names_constraint3.append(name_constraint)

# Objective function
model.setObjective(C.sum('*','*','*'), GRB.MINIMIZE)

model.optimize()

Path("ilp-models").mkdir(exist_ok=True)
model.write(f"ilp-models/{NAME_MSA}.lp")

In [ ]:
"constraint3(1,2,0,3)-(0,1,1,5)" in names_constraint3

In [ ]:
solution_C = model.getAttr("X", C)
solution_U = model.getAttr("X",U)
len(solution_C)>0, len(solution_U)>0

In [ ]:
used_blocks = []
for k,v in solution_C.items(): 
    K,i,j=k
    if v > 0:
        used_blocks.append(
            Block(eval(f"({K},)"),i,j, strings_[K,i,j])
        )

In [ ]:
sorted_solution=sorted(used_blocks, key=lambda b: b.i)
len(sorted_solution), sorted_solution

## Graph construction

In [ ]:
from dataclasses import astuple 
list_nodes = []
list_edges = []

for pos1, block1 in enumerate(sorted_solution[:-1]):
    for rel_pos, block2 in enumerate(sorted_solution[pos1+1:]):
        pos2 = rel_pos + pos1 + 1

        nodes, edges = nodes_edges_from_blocks(block1, block2)
        list_nodes.extend(nodes)
        list_edges.extend(edges)

In [ ]:
k,i,j,b=list_edges[0].node1

In [ ]:
_list_nodes = set([(node.K,node.i,node.j,node.label) for node in list_nodes])
_list_nodes

In [ ]:
_list_edges = []
for edge in list_edges:
    node1=edge.node1
    node2=edge.node2

    _list_edges.append(
        ((node1.K,node1.i,node1.j,node1.label),(node2.K,node2.i,node2.j,node2.label), tuple(edge.seqs))
    )

_list_edges = list(set(_list_edges))

In [ ]:
_list_edges[:3]

In [ ]:
from src.flow.plot_graph import PlotGraph
dg = PlotGraph()
dg(nodes=list_nodes, edges=list_edges, path_save=f"plots/{NAME_MSA}.dot")

In [ ]:
dg.digraph

In [ ]:
list_nodes

In [ ]:
Block((2,),4,5,"GA") in set_B

___

In [ ]:
from collections import defaultdict

# graph in GAF format
lines_gfa = []

HEADER = f"H\t{NAME_MSA}"

# segments (nodes)
lines_segments = [] 
node2id={} # will be useful to add the lines/edges
for id_node, node in enumerate(_list_nodes):
    lines_segments.append(
        f"S\t{id_node}\t{node[-1]}"
    )

    node2id[node]=id_node

# links (edges)
lines_links=[]
data_paths = defaultdict(list)
for edge in _list_edges:
    node1, node2, seqs = edge[0], edge[1], edge[2]
    lines_links.append(
        f"L\t{node2id[node1]}\t+\t{node2id[node2]}\t+\t0M"
    )

    # paths
    for seq in seqs:
        data_paths[seq].extend([node1,node2])

lines_paths=[]
paths = dict()
for seq, nodes_seq in data_paths.items():
    nodes_seq = list(set(nodes_seq))
    id_nodes = [node2id[node] for node in sorted(nodes_seq, key=lambda node: node[1])]
    paths[seq] = ",".join([str(id_node)+"+" for id_node in id_nodes])
    lines_paths.append(
        f"P\tseq{seq}\t\t{paths[seq]}"
    )
lines_gfa.append(HEADER)
lines_gfa.extend(list(set(lines_segments)))
lines_gfa.extend(list(set(lines_links)))
lines_gfa.extend(lines_paths)

path_gfa = Path(f"../experiment/gfa/{NAME_MSA}.gfa")
path_gfa.parent.mkdir(exist_ok=True, parents=True)
with open(path_gfa,"w") as fp:
    for line in lines_gfa:
        fp.write(line + "\n")



In [ ]:
# bandage labels
bandage_labels(path_gfa, path_save_labels=f"../experiment/gfa/{NAME_MSA}_bandage_labels.csv")


In [ ]:
_list_edges

In [ ]:
_list_nodes

In [1]:
# Allow src folder to be imported from this notebook
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import json
from src.blocks import Block
from src.ilp.input import InputBlockSet
from src.ilp.optimization import Optimization
from src.ilp.variaton_graph_parser import asGFA
NAME_MSA = "toyexample"
# Load set of decomposed blocks
path_blocks = f"../experiment/block_decomposition/{NAME_MSA}.json"
path_msa=f"../msas/{NAME_MSA}.fa"

with open(path_blocks) as fp:
    blocks = [Block(*block) for block in json.load(fp)] 

inputset_gen = InputBlockSet()
inputset = inputset_gen(path_msa, blocks)

opt = Optimization(blocks=inputset, path_msa=path_msa)
opt_coverage = opt()

parser=asGFA()
parser(opt_coverage,path_gfa=f"../experiment/gfa/{NAME_MSA}.gfa")

[Block(K=(0,), i=0, j=0, label='A'), Block(K=(2,), i=4, j=4, label='G')]
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 1026 rows, 43 columns and 2078 nonzeros
Model fingerprint: 0x56c8bd8f
Variable types: 0 continuous, 43 integer (43 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 1026 rows and 43 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 64 available processors)

Solution count 2: 6 16 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000000e+00, best bound 6.000000000000e+00, gap 0.0000%
Not consecutive blocks
Condicion- consecutive blocks
Not consecuti

In [ ]:
# constraint = model.getConstrByName("constraint2(1,28)")
# print(f"{model.getRow(constraint)} {constraint.Sense} {constraint.RHS}")